<a href="https://colab.research.google.com/github/elizabethl03/AI-Signature-Recognition/blob/main/SignatureDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# imports

import numpy as np
from skimage.filters import threshold_otsu # for converting to binary & image processing
from scipy import ndimage # image processing for conversion to binary
import matplotlib.image as mpimg # use to read image into array in preprocessing
import matplotlib.cm as cm
import matplotlib
import matplotlib.pyplot as plt
import pandas as ps
import tensorflow
import os

from google.colab import drive

import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm
from scipy import ndimage
from skimage.measure import regionprops
from skimage import io
from skimage.filters import threshold_otsu   # For finding the threshold for grayscale to binary conversion
import tensorflow as tf
import pandas as pd
import numpy as np
from time import time
import keras
from tensorflow.python.framework import ops
import tensorflow.compat.v1 as tf

In [35]:
real_sigs = '/content/drive/My Drive/elink-CSE30124-Fall2023-submissions/Final Project/Signatures/Real'
forged_sigs = '/content/drive/My Drive/elink-CSE30124-Fall2023-submissions/Final Project/Signatures/Forged'

In [17]:
# normalize color and convert to greyscale to account for different color signatures

def convertcolor(img):
  g = np.zeros((img.shape[0], img.shape[1]))

  for row in range(len(image)):
    for col in range(len(image[row])):
      g[row][col] = np.average(img[row][col])

  return g

# make gray image, and translate to binary
def scale(img):
  blur = 0.8

  img = ndimage.gaussian_filter(img, blur)

  t = threshold_otsu(img)
  binary = img > t
  binary = np.logical_not(binary)
  return binary


In [18]:
# pre process the binary image

def process_img(path, img=None, display=True):

  if img is None:
    img = mpimg.imread(path) # use matplotlib to read img into array

  if display == True:
    plt.imshow(img)
    plt.show()
  grey = convertcolor(img)
  if display:
    plt.imshow(grey, cmap = matplotlib.cm.Greys_r)
    plt.show()

  row, col = np.where(bin == 1)

  sign = bin[row.min(): row.max(), col.min(), col.max()]

  if display:
    plt.imshow(sign, cmap = matplotlib.cm.Greys_r)
    plt.show()
  return sign

In [19]:
# get features

def Ratio(img):
  a = 0

  for row in range(len(img)):
    for col in range(len(img[0])):
      if img[row][col]:
        a = a + 1
  total = img.shape[0] * img.shape[1]

  return a / total

In [20]:
# get centroid

def Centroid(img):
  num = 0

  a = np.array([0,0])
  for row in range(len(img)):
    for col in range(len(img[0])):
      if img[row][col]:
        b = np.array([row, col])
        a = np.add(a, b)
        num += 1

  rc = np.array([img.shape[0], img.shape[1]])

  centroid = a / num
  centroid = centroid / rc

  return centroid[0], centroid[1]


In [21]:
def eccentricitysolidity(img):
  r = regionprops(img.astype('int8'))

  return r[0].eccentricity, r[0].solidity

In [22]:
# analyze the skewing of data

def SkewKurtosis(img):
  h, w = img.shape
  x = range(w)
  y = range(h)

  # projections on x & y axes
  xp = np.sum(img, axis=0)
  yp = np.sum(img, axis=1)

  # centroid
  cx = np.sum(x*xp) / np.sum(xp)
  cy = np.sum(y*yp) / np.sum(yp)

  #st dev
  x2 = (x - cx)**2
  y2 = (y - cy)**2

  sx = np.sqrt(np.sum(x2*xp)/np.sum(img))
  sy = np.sqrt(np.sum(y2*yp)/np.sum(img))

  #skew

  x3 = (x - cx) ** 3
  y3 = (y - cy) ** 3

  skewx = np.sum(xp * x3) / (np.sum(img) * sx**3)
  skewy = np.sum(yp * y3) / (np.sum(img) * sy**3)

  # Kurtosis
  x4 = (x - cx) ** 4
  y4 = (y - cy) ** 4

  # normal distribution

  kurtx = np.sum(xp*x4) / (np.sum(img) * sx**4) - 3
  kurty = np.sum(yp*y4) / (np.sum(img) * sy**4) - 3

  return (skewx, skewy), (kurx, kurty)

In [23]:
def GetFeatures(path, img=None, display=False):
  if img is None:
    img = mpimg.imread(path)
  img = process_img(path, display=display)
  ratio = Ratio(img)
  centroid = Centroid(img)
  eccentricity, solidity = eccentricitysolidity(img)
  skewness, kurtosis = SkewKurtosis(img)
  retVal = (ratio, centroid, eccentricity, solidity, skewness, kurtosis)

  return retVal

In [24]:
def CSVFeatures(path, img=None, display=False):
  if img is None:
    img = mpimg.imread(path)

  temp = GetFeatures(path, display=display)

  features = (temp[0], temp[1][0])

  return features

In [33]:
#save feature

def makeCSV():
  drive.mount('/content/drive')

  features_path = '/content/drive/My Drive/elink-CSE30124-Fall2023-submissions/Final Project/Features'
  training_path = '/content/drive/My Drive/elink-CSE30124-Fall2023-submissions/Final Project/Features/Training'
  testing_path  = '/content/drive/My Drive/elink-CSE30124-Fall2023-submissions/Final Project/Features/Testing'

  if not os.path.exists(features_path):
    # If the folder doesn't exist, create it
    os.makedirs(features_path)
  if not os.path.exists(training_path):
    # If the folder doesn't exist, create it
    os.makedirs(training_path)
  if not os.path.exists(testing_path):
    # If the folder doesn't exist, create it
    os.makedirs(testing_path)


  # real signatures
  gpath = real_sigs
  fpath = forged_sigs

  for person in range(1, 4):
    per = ('00'+str(person))[-3:]
    print('Saving features for person id-', per)

    with open('training url') as handle:
      handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')
      for i in range(0,3):
        source = os.path.join(gpath, per+per+'_00'+str(i)+'.png')
        features = CSVFeatures(path=source)
        handle.write(','.join(map(str, features))+',0\n')
      for i in range(0,3):
        source = os.path.join(fpath, '021'+per+'_00'+str(i)+'.png')
        features = getCSVFeatures(path=source)
        handle.write(','.join(map(str, features))+',0\n')

      with open('url...'):
        handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')
        for i in range(3, 5):
          source = os.path.join(gpath, per+per+'_00'+str(i)+'.png')
          features = getCSVFeatures(path=source)
          handle.write(','.join(map(str, features))+',1\n')
        for i in range(3,5):
          source = os.path.join(fpath, '021'+per+'_00'+str(i)+'.png')
          features = getCSVFeatures(path=source)
          handle.write(','.join(map(str, features))+',0\n')




In [ ]:
def testing(path):
  feature = CSVFeatures(path)
  if not(os.path.exists('path...')):
    os.mkdir('dir...')

  with open('path...') as handle:
    handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y\n')
    handle.write(','.join(map(str, feature))+'\n')

In [ ]:
n_input = 9
train_person_id = input("Enter person's id: ")
test_image_path = input("Ener path of signature image: ")
train_path = '....'
testing(tes_image_path)
test_path = '...'

def readCSV(train_path, test_path, type2=False):
  df = pd.read_csv(train_path, usecols=range(n_input))
  train_input = np.array(df.values)
  train_input = train_input.astype(np.float32, copy=False)
  df = pd.read_csv(train_path, usecols=(n_input,))
  temp = [elem[0] for elem in df.values]
  correct = np.array(temp)
  corr_train = keras.utils.to_categorical(correct, 2)

  df = pd.read_csv(test_path, usecols=range(n_input))
  test_input = np.array(df.values)
  test_input = test_input.astype(np.float32, copy=False)
  if not(type2):
    df = pd.read_csv(test_path, usecols=(n_in))